![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/SENTIMENT_EN_FINANCE.ipynb)


# **Sentiment Analysis of Financial news**

## 1. Colab Setup

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [3]:
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


## 2. Some sample examples

In [4]:
text_list = [
"""In April 2005 , Neste separated from its parent company , Finnish energy company Fortum , and became listed on the Helsinki Stock Exchange .""",
"""Finnish IT solutions provider Affecto Oyj HEL : AFE1V said today its slipped to a net loss of EUR 115,000 USD 152,000 in the second quarter of 2010 from a profit of EUR 845,000 in the corresponding period a year earlier .""",             
"""10 February 2011 - Finnish media company Sanoma Oyj HEL : SAA1V said yesterday its 2010 net profit almost tripled to EUR297 .3 m from EUR107 .1 m for 2009 and announced a proposal for a raised payout .""",
"""Profit before taxes decreased by 9 % to EUR 187.8 mn in the first nine months of 2008 , compared to EUR 207.1 mn a year earlier .""",
"""The world 's second largest stainless steel maker said net profit in the three-month period until Dec. 31 surged to euro603 million US$ 781 million , or euro3 .33 US$ 4.31 per share , from euro172 million , or euro0 .94 per share , the previous year .""",
"""TietoEnator signed an agreement to acquire Indian research and development ( R&D ) services provider and turnkey software solutions developer Fortuna Technologies Pvt. Ltd. for 21 mln euro ( $ 30.3 mln ) in September 2007 .""",
]

## 3. Define Spark NLP pipeline

In [6]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

embeddings = BertSentenceEmbeddings\
    .pretrained('sent_bert_wiki_books_sst2', 'en') \
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

sentimentClassifier = ClassifierDLModel.pretrained("classifierdl_bertwiki_finance_sentiment", "en") \
  .setInputCols(["sentence_embeddings"]) \
  .setOutputCol("class_")

financial_sentiment_pipeline = Pipeline(
    stages=[document, 
            embeddings, 
            sentimentClassifier])

sent_bert_wiki_books_sst2 download started this may take some time.
Approximate size to download 389.7 MB
[OK!]
classifierdl_bertwiki_finance_sentiment download started this may take some time.
Approximate size to download 22.5 MB
[OK!]


## 4. Run the pipeline

In [7]:
df = spark.createDataFrame(text_list, StringType()).toDF("text")
result = financial_sentiment_pipeline.fit(df).transform(df)

## 5. Visualize results

In [9]:
result.select(F.explode(F.arrays_zip(result.document.result, 
                                     result.class_.result)).alias("cols")) \
      .select(F.expr("cols['1']").alias("class"),
              F.expr("cols['0']").alias("document")).show(truncate=False)

+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class   |document                                                                                                                                                                                                                                                   |
+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|neutral |In April 2005 , Neste separated from its parent company , Finnish energy company Fortum , and became listed on the Helsinki Stock Exchange .                                                             